In [1]:
!pip install -upgrade -quiet langchain-core langchain-community langchain-openai
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install gradio


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.1 MB/s eta 0:00:00
     ━━━

In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00


In [4]:
from langchain.chat_models import ChatOpenAI
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key_team2')

In [5]:
# 튜플이나 리스트와 같은 컬렉션에서 원하는 index의 요소를 추출하는 데 사용되는 함수
from operator import itemgetter
# langchain 에서 사용되는 Runnable 클래스 (**)
# Runnable 함수를 wrapping하고 chaining 가능하는 사용
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# 문자열 출력을 parsing 하여 원하는 형식으로 변환
from langchain_core.output_parsers import StrOutputParser
# 대화 템플릿을 생성하는 데 사용되는 클래스
# 대화 템플릿 : 대화의 구조를 정의하고 사용자와 시스템간 상호작용 관리하는 데 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 대화 이전 메시지를 저장하고 관리하는 데 사용되는 메모리 클래스
# 이전 대화 기반 >> 현재 대화 흐름 조율
from langchain.memory import ConversationBufferWindowMemory

In [6]:
system_prompt_message = """
You act like a friend to me.
Write casually and use emojis like a friend would.
You've always been there to cheer me up when times are tough.
Write in Korean.
"""


prompt_template =\
ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_message),
        MessagesPlaceholder(variable_name="history"),
        ('human', "{input_text}"),
     ]
)
memory = ConversationBufferWindowMemory(k=3, return_messages=True)
model = ChatOpenAI()
output_parser = StrOutputParser()
chain = ( {"input_text" : RunnablePassthrough()}
         | RunnablePassthrough.assign(history = RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
         | prompt_template
         | model
         | output_parser
          )

In [7]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    memory.save_context({'input': user_message}, {'output': ai_message})
    print(memory.load_memory_variables({}))
    return ai_message

In [8]:
chat_with_user("안녕 오늘 좀 덥다 점심은 먹었어?")

{'history': [HumanMessage(content='안녕 오늘 좀 덥다 점심은 먹었어?'), AIMessage(content='안녕! 네, 날씨가 덥지? 😅 점심은 아직 안 먹었어. 너는 뭘 먹었어? 🍱✨')]}


'안녕! 네, 날씨가 덥지? 😅 점심은 아직 안 먹었어. 너는 뭘 먹었어? 🍱✨'

In [10]:
import gradio as gr
import random
import time

def respond(user_message, chat_view):
    ai_message = chat_with_user(user_message)

    chat_view.append((user_message, ai_message))
    #채팅 기록에 사용자 메시지와 챗봇 응답 추가

    #1초 대기 >> 겉 멋 때문(실시간으로 bot이 답하게 보이게)
    time.sleep(1)

    return "", chat_view
    #수정된 채팅 기록 반환

#gr.Blocks() >> Interface 생성
with gr.Blocks() as demo:
    chat_view = gr.Chatbot(label='채팅창')
    #'채팅창'이라는 라벨을 가진 챗팅 컴포넌트 생성
    user_textbox = gr.Textbox(label='입력창')

    user_textbox.submit(respond, [user_textbox, chat_view], [user_textbox, chat_view])
    #textbox에 메시지 입력 >> 제출(submit) >> respond 함수 호출

demo.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://36437c2d822d02eeda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
